In [1]:
import numpy as np

# data I/O
data = open('input.txt', 'r').read() # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

data has 1115393 characters, 65 unique.


In [2]:
# hyperparameters
hidden_size = 512 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-2

In [3]:
# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) #hidden bias
by = np.zeros((vocab_size, 1)) #output bias 
# so we do expect vector presented as (x_size, 1) -> basically long column, and weight are just long rows 

Here’s the **short, simple derivation** for the gradient of the **softmax cross-entropy loss** w.r.t. the logits \(f_k\). We want:

## 1. Take the Derivative $\frac{\partial L_i}{\partial f_k}$

1. **Derivative of $-f_{y_i}$** w.r.t. $f_k$:
   $$
   \frac{\partial}{\partial f_k}(-f_{y_i}) 
   = \begin{cases}
   -1, & \text{if } k = y_i, \\
   0, & \text{otherwise}.
   \end{cases}
   $$
   That’s $-\mathbf{1}\{y_i = k\}$.

2. **Derivative of $\log(\sum_{j}\exp(f_j))$** w.r.t. $f_k$:
   $$
   \frac{\partial}{\partial f_k} \log\Bigl(\sum_{j} \exp(f_j)\Bigr)
   = \frac{\exp(f_k)}{\sum_{j}\exp(f_j)}
   = p_k.
   $$

**Add them up**:
$$
\frac{\partial L_i}{\partial f_k}
= -\mathbf{1}\{y_i = k\} \;+\; p_k
= p_k \;-\; \mathbf{1}\{y_i = k\}.
$$

---

## 3. Interpretation

- $p_k$ is the **predicted** probability for class \(k\).
- $\mathbf{1}\{y_i=k\}$ indicates the **true** class.
- Hence, if the model incorrectly boosts \(p_k\), the gradient pushes \(f_k\) (the logit) **down**. If \(k = y_i\), it pushes \(f_{y_i}\) **up**, correcting the model’s predictions.

That’s the simplest chain‑rule derivation showing why:

$$
\boxed{\frac{\partial L_i}{\partial f_k}
= p_k - \mathbf{1}\{y_i = k\}.}
$$

In [4]:
def lossFun(inputs, targets, hprev):
  """
  inputs,targets are both list of integers.
  hprev is Hx1 array of initial hidden state
  returns the loss, gradients on model parameters, and last hidden state
  """
  xs, hs, ys, ps = {}, {}, {}, {}
  hs[-1] = np.copy(hprev)
  loss = 0
  # forward pass
  for t in range(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
    xs[t][inputs[t]] = 1
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
    loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
  # backward pass: compute gradients going backwards
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(range(len(inputs))):
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
    dWhy += np.dot(dy, hs[t].T)
    dby += dy
    dh = np.dot(Why.T, dy) + dhnext # backprop into h
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
    dbh += dhraw
    dWxh += np.dot(dhraw, xs[t].T)
    dWhh += np.dot(dhraw, hs[t-1].T)
    dhnext = np.dot(Whh.T, dhraw)
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

In [5]:
def sample(h, seed_ix, n):
  """ 
  sample a sequence of integers from the model 
  h is memory state, seed_ix is seed letter for first time step
  """
  x = np.zeros((vocab_size, 1))
  x[seed_ix] = 1
  ixes = []
  for t in range(n):
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by
    p = np.exp(y) / np.sum(np.exp(y))
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    x = np.zeros((vocab_size, 1))
    x[ix] = 1
    ixes.append(ix)
  return ixes

In [6]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0
while True:
    # prepare inputs (we're sweeping from left to right in steps seq_length long)
    if p+seq_length+1 >= len(data) or n == 0: 
        hprev = np.zeros((hidden_size, 1)) # reset RNN memory
        p = 0 # go from start of data
        
    inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
    targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length + 1]]

    #sample from the model now and then 
    if n % 100 == 0:
        sample_ix = sample(hprev, inputs[0], 200)
        txt = ''.join(ix_to_char[ix] for ix in sample_ix)
        print('----\n %s \n----' % (txt, )) 
    
    # forward seq_length characters through the net and fetch gradient
    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001
    if n % 100 == 0: print('iter %d, loss: %f' % (n, smooth_loss)) # print progress

    # perform parameter update with Adagrad
    for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
        mem += dparam * dparam
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update
    
    p += seq_length # move data pointer
    n += 1 # iteration counter 

    if n == 44700 * 10: # 10 times pass the full text 
        break


----
 ip:PK?MMAsWkNSQBycK'GHkD&:eH,Y!VUtCvRoX$vipZkufPYLdNtWlvPTEwNxIvEGDtfnvTdgT
ajUFNZLlLz;o:.X3XWZt&wP??butSjzD
yqMMNY
,gUfJAk3EMiB; kxrBvMR-Z$;f HhZLCRBfx;iWX&:;TJf?hWL
H;
!qNVkkFDr:tfcSk:x.wTGWmAGxUPu; 
----
iter 0, loss: 104.359719
----
 :Ni nhss si,sot,itNotoo

Fit psb,sa, a!i  thwi lht oonr
h,rws iewevthCiyt ottw' iilheSou ah tnliai  nhfirweto
hniynlith , gNtneueeehvheeot;thhthteyhhsl
nhi:h a l or.matCptoo epn naial erhlht bsl 
tcdi 
----
iter 100, loss: 102.893858
----
 lnayreo ,!kovanli cnc Cearuisebe.bneSnpomlsai'eesosprmlo u.ew htl eiynsileolk lnaseaYTphasyywcpueea!wa,kiraEluu -lelnnbmkpogec
h: a t sae,r tmoy otlynlr rd t-uetw nr
 Utbfmtani nhyi
ms Fo'u
rn n:omp   
----
iter 200, loss: 101.019649
----
 nn t  o!ltodr
 ol
 y
  tvierl nt?lhlsaleif nIeuaataynWdrnh  EtFhb
tviS
sue
hahfh bf a
sM walmss
 t 


sam
!Itry I

pbts SkyWthw:
hAeosaim rrordtoHoun:
rebsclivsegrewt etl'l
eocueoimnufsib U ye-zei dwo 
----
iter 300, loss: 99.391705
----
 wibkewgeyunnttuoty  
hyhrLetoscee  
i

In [7]:
sample(hprev, 1, 200)
txt = ''.join(ix_to_char[ix] for ix in sample_ix)
print('----\n %s \n----' % (txt, )) 
    

----
 now it will Qourd. Whewery carre of your latser'bloh!

FORDINAND:
As I me, becain'd inno lish iflemlasch's soveening and would?

VERCIRIA:
I world eftremanth and bown, in.

MIRCILILA:
The grisir do he 
----
